# StyleGAN2

Reference: https://github.com/NVlabs/stylegan2

My notebooks on Github:
-   https://github.com/woctezuma/steam-stylegan2
-   https://github.com/woctezuma/steam-stylegan

## Installing StyleGAN2

In [5]:
!rm -rf stylegan2/
!git clone https://github.com/NVlabs/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


In [6]:
%cd stylegan2/

/content/stylegan2


In [7]:
!nvcc test_nvcc.cu -o test_nvcc -run

CPU says hello.
GPU says hello.


## Mounting Google Drive

In [0]:
!pip install Google-Colab-Transfer

In [0]:
import colab_transfer

colab_path = colab_transfer.get_path_to_home_of_local_machine()
drive_path = colab_transfer.get_path_to_home_of_google_drive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
colab_transfer.mount_google_drive()

## Data parameters

In [0]:
folder_name = 'datasets/steam/'

## Preparing datasets (once)

### Importing data from Google Drive

Data consists of vertical Steam banners, resized to square resolution (256x256 or 128x128).

In [3]:
colab_transfer.copy_file(file_name='128x128.zip')

Copying /content/drive/My Drive/128x128.zip to /content/128x128.zip


In [0]:
!unzip 128x128.zip -d /content/

### Preparing data for StyleGAN2

In [0]:
!python dataset_tool.py create_from_images '/content/datasets/steam' '/content/128x128'

Loading images from "/content/128x128"
Creating dataset "/content/datasets/steam"
Added 14035 images.


### Archive prepared datasets to Google Drive

In [0]:
colab_transfer.copy_folder(folder_name,
                           source=colab_path,
                           destination=drive_path)

Copying /content/datasets/steam/ to /content/drive/My Drive/datasets/steam/


## Importing prepared datasets from Google Drive

In [11]:
colab_transfer.copy_folder(folder_name,
                           source=drive_path,
                           destination=colab_path)

Copying /content/drive/My Drive/datasets/steam/ to /content/datasets/steam/


## Training networks

In [0]:
!python run_training.py --data-dir='/content/datasets' --dataset=steam --config=config-e --metrics=none

Local submit - run_dir: results/00001-stylegan2-steam-1gpu-config-e
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x7110000 @  0x7f43aeae3001 0x7f43ab5690f5 0x7f43ab5cc4d9 0x7f43ab5ce6af 0x7f43ab6649b8 0x50ac25 0x50d390 0x508245 0x509642 0x595311 0x54a6ff 0x551b81 0x5a067e 0x50d966 0x508245 0x58958c 0x5a067e 0x50d966 0x508245 0x58958c 0x5a067e 0x50d966 0x509d48 0x50aa7d 0x50c5b9 0x509d48 0x50aa7d 0x50c5b9 0x508245 0x58958c 0x5a067e
tcmalloc: large alloc 4294967296 bytes == 0x7f41f7c26000 @  0x7f43aeae11e7 0x7f43ab568f71 0x7f43ab5cc55d 0x7f43ab5ccbe7 0x7f43ab664898 0x50ac25 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245 0x50a080 0x50aa7d 0x50d390 0x509d48 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50c5b9 0x508245 0x50a080 0x50aa7d 0x50d390 0x508245
tcmalloc: large alloc 4294967296 bytes == 0x7f40f6c24000 @  0x7f43aeae1

## Generating images

### Using the first snapshot

In [0]:
# Generate 1000 random images without truncation
!python run_generator.py generate-images --seeds=0-999 --truncation-psi=1.0 \
  --network=results/00000-stylegan2-steam-1gpu-config-e/network-snapshot-000000.pkl

### Using the last snapshot

In [0]:
# Generate 1000 random images without truncation
!python run_generator.py generate-images --seeds=0-999 --truncation-psi=1.0 \
  --network=results/00000-stylegan2-steam-1gpu-config-e/networks-final.pkl